# 第３章

3.1 特徴点（キーポイント）

特徴点（キーポイント）とは、点群の中でも特徴的なパターンを持ち、他の点との区別が付きやすいランドマークになりうる点のことを指します。例えば、第4章で紹介する点群レジストレーション（位置合わせ）を考えましょう。点群レジストレーションを行うためには、複数個の異なる視点から撮影した複数の点群データに対して、ある点群データから別の点群データへの座標変換を求めることになります。このとき、点群データ間でどの点とどの点が対応するかを求める（対応点探索をする）必要があります。ここで、一方の点群データ内のすべての点に対する対応点をもう一方の点群データから探すことは計算量の観点から現実的ではありません。さらに、多くの特徴的でない点に対する誤対応点の存在によって点群レジストレーションの精度が下がるかもしれません。以上の理由から、点群レジストレーションの最初の段階で特徴点の抽出が行われます。

本節では2つの特徴点抽出手法を紹介します。まず、Harris3D について述べます。これは2次元画像処理でよく使われる Harris コーナー検出[24]の拡張手法です。次に、Intrinsic Shape Signature (ISS) [84]を紹介します。これはOpen3D をはじめとする多くのオープンソースライブラリに実装されていることから、3次元点群処理でもよく使われる特徴点抽出手法です。

3.1.1 Harris3D

Harris3D は2次元画像処理でよく使われる Harris コーナー検出[24]の拡張手法です。まず、Harris コーナー検出について簡単に説明します。コーナーとなる点は、その点から全方向に対する画素値の変化が大きい点となります。とある窓関数 w(u, v) と画素値 I(u, v) を用いて、とあるピクセルシフト量 (x, y) に対する画素値変化量 E(x, y) は次式で表せます。

```
E(x, y) = Σ w(u, v)|I(x + u, y + v) - I(u, v)|²   (3.1)
```

ただし、全方向を考えるので、(x, y) の集合は {(1, 0), (1, 1), (0, 1), (-1, 1)} とします。窓関数 w(u, v) は局所領域を定義するためのものです。ある大きさの矩形領域でもよいし、滑らかな関数の窓を用いるためのガウス関数 w(u, v) = exp - (u² + v²)/2σ² とすることもあります。ここで、x, y 方向の画素の勾配をそれぞれ Ix, Iy とおき、ピクセルシフト量 (x, y) が十分に小さい値であるとしてテイラー展開を用いると、E(x, y) は式で近似できます。

```
E(x, y) ≈ [x y]M[x/y], M = Σ w(u,v)[IxIx IxIy; IxIy IyIy]   (3.2)
```

行列 M の2つの固有値（仮に λ₁ と λ₂ とします）と点の画素値との間には下記の関係があります。まず、λ₁ と λ₂ が両方小さい場合、すべての方向へのシフトに対して画素値の変化が小さいことになります。次に、λ₁ と λ₂ のどちらか一方が大きく、もう一方が小さい場合は、ある方向にのみ画素値の変化が大きい点、すなわちエッジ上の点であると考えられます。最後に、λ₁ と λ₂ が両方大きい場合は、すべての方向へのシフトに対して画素値の変化が大きい点、すなわちコーナー点であると考えられます。以上をふまえて、λ₁ と λ₂ が両方大きい点を発見するために、次式の Harris 指標 R を計算します。

```
R = det(M) - k tr(M)² = λ₁λ₂ - k(λ₁ + λ₂)²   (3.3)
```

ただし k は実験的に決定するパラメータです注1。コーナー点は R が正の大きな値に、エッジ上の点は R が負の値に、そして特に特徴のないフラットな点は R が正の小さな値になります。よって、R があるしきい値を超える点を抽出すればコーナー点を検出できます。なお、k を明示的に設定することを避けるために、式 (3.3) の代わりに次式が用いられることもあります。

```
R = det(M)/tr(M)   (3.4)
```

最後に、各点に対してある範囲の近傍点をチェックし、自身の R 値よりも小さい R 値を持つ点を消去します。これは、画像中のほぼ同じ位置に多くのコーナー点が重複検出されることを防ぐために局所最大となる点のみを抽出する処理であり、一般に Non Maximum Suppression (NMS) と呼ばれています。

以上が2次元画像処理における Harris コーナー検出の流れです。3次元点群処理のオープンソースライブラリである Point Cloud Library (PCL) では、これを3次元処理に拡張した HarrisKeypoint3D 注2 という関数が用意されています。ここでは、画素値勾配の代わりに点の法線ベクトルを用いて Harris 指標を計算しています。Python のサンプルコードを下記に示します。ただし、式 (3.3) の代わりに式 (3.4) を用いています。

## Spherical Harmonics Representation（球面調和関数表現）

やあ！難しそうな質問だけど、できるだけわかりやすく説明していくね。

Spherical Harmonics Representation（球面調和関数表現）は、3次元空間上の関数を表現するための強力な数学的ツールなんだ。特に球面上で定義された関数を扱うときにすごく便利！

簡単に言うと、円周上の関数をフーリエ級数で周期関数の重ね合わせとして表現できるように、球面上の関数を球面調和関数の重ね合わせで表現できるんだ。

## 具体的な説明

1. **基本的なアイデア**
まず、平面上の2次元の場合を考えてみよう。例えば、音の波形は正弦波（サイン関数）とコサイン関数の重ね合わせで表現できるよね。これを3次元の球面に拡張したのが球面調和関数なんだ。

2. **数学的な表現**
球面調和関数 $Y_l^m(\theta,\phi)$ は、以下のように表現されるよ：

$Y_l^m(\theta,\phi) = N_l^m P_l^m(\cos\theta)e^{im\phi}$

ここで：
- $\theta$ は天頂角 (0 から π)
- $\phi$ は方位角 (0 から 2π)
- $l$ は次数 (0, 1, 2, ...)
- $m$ は方位数 (-l から +l)
- $N_l^m$ は正規化係数
- $P_l^m$ はルジャンドル陪関数

3. **実際の応用例**
これ、実は身近なところでも使われてるんだ！例えば：
- 量子力学での電子軌道の表現
- コンピュータグラフィックスでの3D形状の表現
- 地球の重力場のモデリング
- 音場の解析

4. **なぜ便利なの？**
球面調和関数の最大の特徴は、直交性があることなんだ。つまり、異なる $(l,m)$ の組み合わせの球面調和関数同士は互いに独立していて、任意の球面上の関数を一意に分解できるんだ。

これは例えるなら、「色んな音の周波数成分を分解する」みたいなものだね。複雑な形状も、基本的な形状の組み合わせで表現できちゃうわけ！

5. **具体的なイメージ**
低次の球面調和関数は、こんな感じのパターンを作るよ：
- $l=0$: 完全な球
- $l=1$: ダンベルのような形
- $l=2$: より複雑な波打つような形

これらを組み合わせることで、めちゃくちゃ複雑な3D形状だって表現できちゃうんだ！

難しく聞こえるかもしれないけど、要するに「3次元の形を数学的にスマートに表現する方法」って考えるといいかな。

もっと具体的な応用例や、特定の部分について詳しく知りたいことはある？気軽に聞いてね！

もちろん！一つずつ丁寧に説明していくね。

まず、球面調和関数の式をもう一度見てみよう：

$Y_l^m(\theta,\phi) = N_l^m P_l^m(\cos\theta)e^{im\phi}$

## 1. 座標系の説明 ($\theta$ と $\phi$)

まず、3次元空間の点を表す方法として、球座標系というものを使うんだ。

- **天頂角 $\theta$ (シータ)**
  - 真上（z軸）からの角度を表す
  - 0° から 180°（0 から π ラジアン）の範囲
  - 例えると、地球儀で「緯度」みたいなもの
  - ただし、普通の緯度とは少し違って、北極が0°、赤道が90°、南極が180°

- **方位角 $\phi$ (ファイ)**
  - 水平面での角度（x軸からの回転）を表す
  - 0° から 360°（0 から 2π ラジアン）の範囲
  - 地球儀でいう「経度」みたいなもの

## 2. 次数と方位数 ($l$ と $m$)

- **次数 $l$ (エル)**
  - 関数の「複雑さ」を決める整数（0以上の値）
  - $l$ が大きくなるほど、より細かい模様が表現できる
  - 例えば：
    - $l=0$ → 一様な球
    - $l=1$ → 単純な凹凸
    - $l=2$ → より複雑な凹凸

- **方位数 $m$ (エム)**
  - 各次数 $l$ に対して、$-l$ から $+l$ までの整数値をとる
  - 方位方向（経度方向）の波の数を決める
  - 例：$l=1$ のとき、$m$ は -1, 0, +1 の3つの値をとれる

## 3. その他の要素

- **正規化係数 $N_l^m$**
  - 関数の大きさを適切に調整する係数
  - これがないと、関数の値が大きくなりすぎたり小さくなりすぎたりしちゃう
  - 数学的には以下のように定義されるよ：
  
  $N_l^m = \sqrt{\frac{(2l+1)}{4\pi}\frac{(l-m)!}{(l+m)!}}$

- **ルジャンドル陪関数 $P_l^m(\cos\theta)$**
  - 天頂角方向の波形を決める特殊な関数
  - 正直、これの詳しい形は結構複雑で、計算機で計算することが多い
  - でも基本的には「上下方向の波形をどう作るか」を決める関数だと思えばOK！

- **指数関数部分 $e^{im\phi}$**
  - 方位角（経度）方向の周期的な変化を表現
  - オイラーの公式により、実際には以下のような形になる：
  
  $e^{im\phi} = \cos(m\phi) + i\sin(m\phi)$

## 具体例

例えば、一番単純な $l=0, m=0$ の場合：
- $Y_0^0 = \frac{1}{\sqrt{4\pi}}$ （定数）
- これは完全な球を表現する

$l=1, m=0$ の場合：
- $Y_1^0 \propto \cos\theta$
- これは上下方向にグラデーションのかかった球を表現

これらの基本的なパターンを組み合わせることで、複雑な3次元形状を表現できるんだ！

難しい数式が出てきたけど、要するに：
1. 「どの方向に」（$\theta, \phi$）
2. 「どれくらい複雑に」（$l$）
3. 「どんなパターンで」（$m$）
球面上の形を作るかを決める関数、って感じだね。

はい！ルジャンドル陪関数について、もっと詳しく説明していくね。

## ルジャンドル陪関数の基本

ルジャンドル陪関数 $P_l^m(x)$ は、実は「ルジャンドル多項式」という基本的な関数から派生したものなんだ。

まずは簡単な例から見ていこう！

### 低次のルジャンドル多項式 $P_l(x)$
最初の数個のルジャンドル多項式はこんな形になるよ：

- $P_0(x) = 1$
- $P_1(x) = x$
- $P_2(x) = \frac{1}{2}(3x^2 - 1)$
- $P_3(x) = \frac{1}{2}(5x^3 - 3x)$

これらの多項式から、ルジャンドル陪関数が作られるんだ！

## ルジャンドル陪関数の導出

ルジャンドル陪関数 $P_l^m(x)$ は、ルジャンドル多項式を $m$ 回微分することで得られるんだ：

$P_l^m(x) = (-1)^m(1-x^2)^{m/2}\frac{d^m}{dx^m}P_l(x)$

ちょっと複雑に見えるけど、具体例を見ていくと分かりやすいよ！

### 具体例：低次のルジャンドル陪関数

1. **$l=1$ の場合**:
   - $P_1^0(x) = x$
   - $P_1^1(x) = -\sqrt{1-x^2}$

2. **$l=2$ の場合**:
   - $P_2^0(x) = \frac{1}{2}(3x^2 - 1)$
   - $P_2^1(x) = -3x\sqrt{1-x^2}$
   - $P_2^2(x) = 3(1-x^2)$

## 物理的な意味

ルジャンドル陪関数が実際に表現する形を理解するために、球面調和関数での使われ方を見てみよう：

1. **$l=0, m=0$ の場合**:
   - 一様な球を表現
   - $P_0^0 = 1$ なので、どの方向でも同じ値

2. **$l=1$ の場合**:
   - $m=0$: 北極から南極へのグラデーション
   - $m=±1$: x-y平面での双極子パターン

3. **$l=2$ の場合**:
   - より複雑な模様が表現可能
   - 例えば、$m=0$ は「蝶ネクタイ」のような形

## 重要な性質

ルジャンドル陪関数には、いくつかの重要な性質があるんだ：

1. **直交性**:
   異なる $l, m$ の組み合わせの関数は互いに直交する（独立している）

2. **範囲**:
   - $x$ は -1 から 1 の範囲（球面調和関数では $x = \cos\theta$ となる）
   - $m$ は $-l$ から $l$ までの整数値

3. **対称性**:
   - $m$ が偶数のとき、関数は $x$ について偶関数
   - $m$ が奇数のとき、関数は $x$ について奇関数

## 実際の応用例

1. **量子力学**:
   - 水素原子の電子軌道の形を記述
   - 各軌道の角度依存性を表現

2. **重力場の記述**:
   - 地球の重力場の詳細なモデリング
   - 衛星軌道の計算

3. **電磁気学**:
   - 複雑な電場・磁場の表現
   - アンテナの放射パターンの解析

## わかりやすい例え

ルジャンドル陪関数は、「球面上でどんな波形を作るか」を決める「レシピ」みたいなものだと考えるといいかも。例えるなら：

- $l=0$: 平らなケーキ
- $l=1$: 一方向に傾いたケーキ
- $l=2$: 波打つような模様のケーキ

そして $m$ は、その模様が方位角方向にどう変化するかを決めるんだ！

難しい数式が出てきたけど、要するに「3次元空間での波形を作るための数学的な道具」って考えるといいよ。もっと具体的に知りたい部分はある？

なるほど！この部分、3D CADデータの形状特徴を数学的に表現しようとしているところだね。順番に解説していくよ！

## 式の意味を紐解く

1. **基本的な考え方**
まず、3D形状を球面上の関数として表現しているんだ。つまり、物体の形を「どの方向にどれくらい出っ張っているか」という関数として捉えている。

2. **展開式 (3.6) の解釈**
$f(\theta,\phi) = \sum_{l=0}^{\infty}\pi_l(f) = \sum_{l=0}^{\infty}\sum_{m=-l}^{l}a_{lm}Y_l^m(\theta,\phi)$

これは要するに：
- 3D形状を球面調和関数の重ね合わせで表現
- $\pi_l(f)$ は周波数 $l$ の成分を表す
- $a_{lm}$ は各基底関数の係数（重み）

3. **特徴量SH (3.7) の意味**
$\mathrm{SH}(f) = \{\|\pi_0(f)\|, \|\pi_1(f)\|, \ldots\}$

これが本当に大事なところ！この式は：
- 各周波数レベル $l$ でのエネルギー（強さ）を並べたもの
- $\|\pi_l(f)\|$ は周波数 $l$ の成分の大きさ

## 実践的な意味

これを3D CADの特徴量として使う理由は：

1. **回転不変性**
- 物体を回転させても、各周波数成分のエネルギーは変わらない
- つまり、同じ形状は同じような特徴量を持つ

2. **階層的な形状表現**
- 低周波（小さい $l$）：大まかな形状
- 高周波（大きい $l$）：細かいディテール

3. **効率的な比較**
- 形状の類似性を、このエネルギー分布で比較できる
- ベクトル同士の比較で済むので計算が速い

## 具体例

例えば：
```python
# 仮想的なコード例
def compute_shape_descriptor(cad_model):
    # 1. 3D形状を球面関数に変換
    f_theta_phi = convert_to_spherical(cad_model)
    
    # 2. 球面調和関数展開
    coefficients = spherical_harmonics_transform(f_theta_phi)
    
    # 3. 各周波数でのエネルギー計算
    energies = []
    for l in range(max_l):
        energy_l = compute_energy_at_frequency(coefficients, l)
        energies.append(energy_l)
    
    return energies  # これがSH特徴量
```

この特徴量を使うと：
1. 形状検索（似た形状の検索）
2. 形状分類（部品の自動分類）
3. 品質管理（想定との形状の違いチェック）

なんかが効率的にできるようになるんだ！

これで少しクリアになってきた？特に実際の応用面でもっと知りたいところはある？

あ、なるほど！エネルギーという言葉が抽象的だよね。もっと具体的に説明していくよ。

## 周波数とエネルギーの直感的な理解

まず、音の例で考えてみよう：
- 低い音（低周波）と高い音（高周波）が混ざった音があるとき
- それぞれの音の「大きさ」（振幅の二乗）が、その周波数での「エネルギー」
- 全部の周波数のエネルギーを並べると、その音の特徴が分かる

3D形状でも同じような考え方ができるんだ：

## 3D形状での具体例

簡単な例で見てみよう：

1. **完全な球**
```
エネルギー分布：
l=0: ■■■■ (大きい)  // 球の基本的な大きさ
l=1: □ (ほぼ0)      // 偏りがないので
l=2: □ (ほぼ0)      // 高周波成分はほとんどない
```

2. **少し歪んだ球**
```
エネルギー分布：
l=0: ■■■■ (大きい)  // 基本の球の大きさ
l=1: ■■ (中程度)    // 少しの歪み
l=2: □ (小さい)     // 細かい変化は少ない
```

3. **複雑な形状（たくさんの凹凸がある物体）**
```
エネルギー分布：
l=0: ■■■■ (大きい)  // 基本サイズ
l=1: ■■ (中程度)    // 大まかな歪み
l=2: ■■ (中程度)    // 
l=3: ■ (それなり)   // 細かい凹凸による
l=4: ■ (それなり)   // 高周波成分
```

## 数学的な計算方法

実際のエネルギー計算は以下のようになる：

$\|\pi_l(f)\| = \sqrt{\sum_{m=-l}^{l}|a_{lm}|^2}$

これは：
1. その周波数 $l$ での全ての係数 $a_{lm}$ を二乗して足し合わせる
2. その平方根をとる

つまり：
- 各周波数でどれくらい「形状変化の成分」があるかを数値化
- 二乗して足し合わせるのは、プラスマイナスを気にしないため

## 実用的な例

例えば、ある部品の品質管理で：
```python
# 理想的な部品の特徴量
ideal_energies = [100, 50, 20, 5, 1]  # 各周波数のエネルギー

# 実際の部品の特徴量
actual_energies = [98, 52, 25, 8, 3]

# 比較
differences = [abs(i-a) for i,a in zip(ideal_energies, actual_energies)]
if max(differences) > threshold:
    print("品質に問題あり：想定より細かい凹凸が多い")
```

このように、形状の「荒さ」や「複雑さ」を数値化して比較できるんだ！

要するに：
1. 各周波数は形状の「細かさ」のレベル
2. エネルギーはその「細かさ」がどれくらい存在するか
3. それらを並べることで、形状の特徴を表現

という感じだね。これで少しイメージが湧いてきた？もっと具体的に知りたいところはある？

# FPFH特徴量について

In [2]:
from pathlib import Path
import open3d as o3d

bath = (
    Path()
    .cwd()
    .parents[0]
    .joinpath("3rdparty/Open3D/examples/test_data/bathtub_0154.ply")
)

In [ ]:
filename = str(bath)
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10)
)

fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd, search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100)
)

print(fpfh)
print(fpfh.data)


Loading a point cloud from /Users/argon/dev/books/3dpcp_book_codes/3rdparty/Open3D/examples/test_data/bathtub_0154.ply
PointCloud with 1494 points.
Feature class with dimension = 33 and num = 1494
Access its data via data member.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]




もちろん！FPFHについてわかりやすく説明するね。これは3D点群データから特徴量を抽出する手法なんだ。順を追って説明していくよ！

### 1. まず前提として
Point Feature Histograms (PFH)という基本的な手法があって、FPFHはそれを高速化した版なんだ。

### 2. 基本的な考え方
ある3D空間上の点pについて、その周りの形状を数値化して特徴量にしたいんだ。具体的には：

1. 点pの周りのk個の近い点（近傍点）を見つける
2. それらの点の組み合わせについて、3つの値（α, φ, θ）を計算する
3. これらの値をヒストグラム化して特徴量とする

### 3. 3つのパラメータの意味
それぞれのパラメータは、2つの点i, jとその法線ベクトルni, njから計算されるんだ：

1. **α (アルファ)**
```
α = (pj - pi) × ni・nj
```
これは2点間のベクトルと法線ベクトルの関係を表す角度だよ

2. **φ (ファイ)**
```
φ = ni・(pj - pi) / ||pj - pi||
```
これは法線ベクトルと2点間の距離の関係を表すんだ

3. **θ (シータ)**
```
θ = arctan(ni × ((pj - pi) × ni)・nj, ni・nj)
```
これは2つの法線ベクトル間の捻れ具合を表している

### 4. FPFHの特徴
- PFHより計算が速い（高速化版）
- 点の周りの形状を効率的に特徴量化できる
- 3D物体認識や位置合わせなどに使える

### 5. なぜこれが便利なの？
例えば、テーブルの角とテーブルの平らな部分では、周りの点の配置が全然違うよね。FPFHを使うと、そういった「形の特徴」を数値として表現できるんだ。これによって：

- 物体の特徴的な部分を見つけられる
- 似た形状を探せる
- 3D点群の位置合わせができる

### 6. 具体例
テーブルの角を例に取ると：
- 周りの点は90度近い角度で配置されている
- 法線ベクトルも急激に変化する
- これらの特徴がα, φ, θのヒストグラムに反映される

どう？少しはイメージが湧いてきたかな？もし特定の部分でまだ分かりにくいところがあれば、もっと詳しく説明するよ！

In [ ]:
import open3d as o3d
import numpy as np


def extract_fpfh(filename: str, voxel_size: float = 0.05):
    print(" ", filename)

    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    )

    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100)
    )

    sum_fpfh = np.sum(np.array(fpfh.data), 1)

    return sum_fpfh / np.linalg.norm(sum_fpfh)